<a href="https://colab.research.google.com/github/gosaitos/GEN-AI/blob/main/Diffusion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install diffusers transformers accelerate torch safetensors opencv-python

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os
import cv2

# -------------------------------------
# ✅ Load Free Anime Model (No Login Needed)
# -------------------------------------
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16
).to("cuda")

# -------------------------------------
# 🔹 Output Directory for Frames
# -------------------------------------
output_dir = "anime_frames"
os.makedirs(output_dir, exist_ok=True)

# -------------------------------------
# 🔹 Prompt & Generation Settings
# -------------------------------------
prompt = "a cinematic anime landscape with a girl in the wind, cherry blossoms, ultra detailed, by Makoto Shinkai"
num_frames = 10
guidance_scale = 7.5
height, width = 512, 512

# -------------------------------------
# 🔹 Generate Frames
# -------------------------------------
print("🎨 Generating anime-style frames...")
for i in range(num_frames):
    image = pipe(
        prompt,
        guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_inference_steps=30,
        generator=torch.manual_seed(i)
    ).images[0]

    image.save(f"{output_dir}/frame_{i:04d}.png")
    print(f"✅ Frame {i+1}/{num_frames} saved")

# -------------------------------------
# 🔹 Create Video from Frames
# -------------------------------------
def create_video_from_frames(folder, output_path, fps=10):
    print("🎞️ Stitching video...")
    images = sorted([img for img in os.listdir(folder) if img.endswith(".png")])
    frame_array = [cv2.imread(os.path.join(folder, img)) for img in images]

    height, width, _ = frame_array[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for frame in frame_array:
        out.write(frame)
    out.release()
    print(f"🎬 Done! Video saved as {output_path}")

create_video_from_frames(output_dir, "anime_video.mp4", fps=10)
